# IPython para administración de sistemas Unix

Pueden listarse las funciones dentro de un módulo con la consola de `IPython` usando el **tabulador** cuando se importa el módulo, se escribe por consola y se añade un punto:

```python
import subprocess
subprocess.
```

Para conocer un ejemplo de uso de una de las funciones puede añadirse un `?` detrás de la función como en el ejemplo. PUede obtenerse más ayuda si se introducen dos `??`.

In [15]:
subprocess.getoutput??

Es buena idea cuando se crean scripts de shell con Python envolver todas las funciones que representan un módulo en un `main` de tal manera que se ejecuten secuencialmente todas las funciones. De esta manera se tienen por un lado llamadas individuales a las funciones y por otro el conjunto de todas ellas. A esta función `main` se le denomina **flujo de control**.

In [2]:
import subprocess

# Comando 1
def uname_func():
    
    uname = "uname"
    uname_arg = "-a"
    print("Gathering system informatiokn with %s command:\n" % uname)
    subprocess.call([uname, uname_arg])
    
    
#Comando 2
def disk_func():
    
    diskspace = "df"
    diskspace_arg = "-h"
    print("Gathering diskpace information %s command:\n" % diskspace)
    subprocess.call([diskspace, diskspace_arg])

    
# Función principal que invoca otras funciones.
def main():
    uname_func()
    disk_func()
    
main()

Gathering system informatiokn with uname command:

Gathering diskpace information df command:



## Funciones mágicas

Para listar las funciones mágicas introducir: `lsmagic`. También puede utilizarse el tabulador si se pone por delante el caracter especial `%`. Puede desplegarse una ayuda si se introduce el comando: `magic`.

In [11]:
lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%

Un ejemplo de una función mágica, permite crear `alias` como en el ejemplo:

```python
%alias d ls -F
```

Para desactivarlo, utilizar la función mágica `%unalias`.

In [17]:
%alias d ls -F

Por ejemplo, pueden listarse las funciones y módulos interactivos que se están usando en la sesión de IPython con la **función mágica** `%who`o `%whos` con algo extra de información. Por ejemplo, hasta esta celdilla se han ejecutado otras celdas que contienen información. La lista se muestra en la siguiente celdilla.

In [24]:
%whos

Variable     Type        Data/Info
----------------------------------
disk_func    function    <function disk_func at 0x7f5c94500158>
main         function    <function main at 0x7f5c945001e0>
subprocess   module      <module 'subprocess' from<...>python3.6/subprocess.py'>
uname_func   function    <function uname_func at 0x7f5c945000d0>


Una guía rápida de las funciones mágicas puede sacarse con `%quickref`. Un ejemplo se muestra a continuación.

In [27]:
%quickref

Un ejemplo de alias útil para gestionar un sistema Unix pueden ser, como se dijo antes, los alias. Si, por ejemplo, se quiere utilizar el comando `netstat -plunt` de manera más rápida, puede hacerse u alias como el siguiente:

In [35]:
%alias nss netstat -plunt

In [36]:
nss

(Not all processes could be identified, non-owned process info
 will not be shown, you would have to be root to see it all.)
Active Internet connections (only servers)
Proto Recv-Q Send-Q Local Address           Foreign Address         State       PID/Program name    
tcp        0      0 127.0.0.1:3306          0.0.0.0:*               LISTEN      -                   
tcp        0      0 127.0.0.1:6379          0.0.0.0:*               LISTEN      -                   
tcp        0      0 0.0.0.0:51413           0.0.0.0:*               LISTEN      -                   
tcp        0      0 127.0.0.53:53           0.0.0.0:*               LISTEN      -                   
tcp        0      0 127.0.0.1:631           0.0.0.0:*               LISTEN      -                   
tcp        0      0 127.0.0.1:8888          0.0.0.0:*               LISTEN      31724/python3       
tcp        0      0 127.0.0.1:45177         0.0.0.0:*               LISTEN      -                   
tcp        0      0 127.

Además, puede utilizarse las metodologías **hacer-nada** o **hacer-todo** donde se le pueden pasar o no argumentos a un alias o comando e interpretarse de distinta manera. Teniendo por ejemplo el alias:

In [44]:
%alias achoo echo first: "|%s|", second "|%s|"

In [46]:
achoo foo bar

first: |foo|, second |bar|


Para hacerlo con o sin argumentos:

In [47]:
%alias achoo echo "|%l|"

In [54]:
achoo

||


Otra forma de ejecutar un comando de shell es colocando el símbolo `!` delante. La salida de un comando de shell puede almacenarse en una lista, como por ejemplo.

In [75]:
user = 'nachoaz'
process = 'bash'
l = !ps aux | grep $user | grep $process
l # Imprimir la variable 'l'.

['nachoaz   1901  0.0  0.0  24144  5512 pts/3    Ss+  jul15   0:00 /bin/bash',
 'nachoaz   4305  0.0  0.0  14104  3352 ?        S    00:18   0:00 /bin/bash -c ps aux | grep nachoaz | grep bash',
 'nachoaz   4309  0.0  0.0  15648  1104 ?        S    00:18   0:00 grep bash',
 'nachoaz   8916  0.0  0.0  25100  6404 pts/0    Ss+  jul15   0:00 /bin/bash',
 'nachoaz  13901  0.0  0.0  24064  5520 pts/2    Ss   jul15   0:00 /bin/bash']

## cd y marcadores 

## Bookmarks

Crear marcadores se puede hacer de diferentes maneras. Una de ellas es la siguiente: con la directiva `bookmark t` estando, por ejemplo, en el directorio `/tmp` se crea un marcador en ese directorio.

In [7]:
cd /tmp

/tmp


In [8]:
bookmark t

Otra manera de crear un marcador requiere escribir un poco más.

In [20]:
bookmark h /home/

Pueden listarse los marcadores con la siguiente opción. **Nota:** Los marcadores se conservan entre sesiones de IPython por lo que no es necesario cargarlos de nuevo.

In [6]:
%bookmark -l

Current bookmarks:
my_home -> /home/$(whoami)
t       -> /tmp


Para eliminar un marcador se utiliza el argumento `-d` como sigue:

In [23]:
bookmark -d h

UsageError: %bookmark -d: Can't delete bookmark 'h'


In [25]:
bookmark -l

Current bookmarks:
my_home -> /home/$(whoami)
t       -> /tmp


## dhist

El comando `dhist` no solamente guarda el histórico de la sesión actual sino de sesiones anteriores.

In [1]:
dhist

Directory history (kept in _dh)
0: /home/nachoaz/igarag/programming/python


Puede pasarsele argumentos para especificar, por ejemplo, la salida máxima del comando, haciendo una lista de los últimos 5 elementos de la historia con: `dhist 5` o un rango entre valores con `dhist 3 7` 

## Aplicación de Variable

Hasta ahora había una clara diferencia entre los comandos de shell e IPython pero vamos a ver que se pueden cruzar. Se va a tomar un valor que obtendremos de Python y pasarlo a la Shell. Aunque el ejemplo es "poco realista" dado que es raro crear 10 archivos con la fecha dentro, nos da una idea de la utilidad en el cruce de información entre ambos lenguajes.

In [29]:
# {} = elemento actual de la lista
for i in range(3):
    !date > {i}.txt

In [14]:
!ls -l

!cat 0.txt

total 60
-rw-rw-r-- 1 nachoaz nachoaz 57778 ago  3 13:24 ipython_para_sistemas_unix.ipynb
cat: 0.txt: No such file or directory


Eliminamos los archivos creados en el apartado anterior.

In [36]:
!rm *.txt

## Procesado de cadena

mOtra herramienta muy interesante que ofrece IPython es procesar en cadena el resultado de los comandos de shell del sistema. Si, por ejemplo, se quiere ver el PID de todos los procesos que pertenecen a un usuario se podría hacer de la siguiente manera: 

`ps aux | awk '{if ($1 == "myuser") print $2}'`

Si ahora intentamos repetir el proceso usando IPython hay que hacerlo en una pequeña secuencia que es igualmente legible.

In [17]:
ps = !ps aux
ps.grep('mysuer', prune=False).fields(0, 1, 8)

[]

El argumento `prune` se utiliza para que el retorno del argumento sea lo contrario a lo que se quiere o lo que se quiere. Por ejemplo, si quisieramos que el retorno fuera 'todos los usuarios que no sean el nuestro' (`root` por ejemplo) se pondría: `ps.grep('myuser', prune=True)`, es decir, __excluir__ del filtrado. De esa manera devuelve todos los usuarios que no sean `myuser`.

Con el método `fields()` se le especifican a `grep` las columnas que queremos devolver una vez se ejecute el comando. Hace las veces cuando se imprime una columna con `awk` utilizando `$1`, `$2`. En el ejemplo anterior, las columnas `0`, `1` y `8` hacen referencia a `USERNAME`, `PID` y `STARTTIME`.

Otro elemento muy interesante es, seguido del método `fields()` que va seguido del método `grep()` y que ambos devuelven objetos del mismo tipo, se puede añadir el elemento `s` que accede directamente a la lista en el proceso. Esto proporciona una cadena de PID separados por espacio con la que la Shell ya puede trabajar y de podría, por ejemplo hacer `kill $pids` desde dentro de IPython.

Por último, el método `grep()` adopta un último parámetro opcional denominado `field`. Si se especifica, los criterios de búsqueda tienen que coincidir con ese campo para que se incluya en el resultado. Esto coincidía con las dilas exactas que se quieren pero mostraba toda la fila. Para llegar solo al PID hay que añadirle `fields(1)` detrás.

In [25]:
ps = !ps aux
ps.grep('nachoaz', field=0).fields(1)

['2418',
 '2419',
 '2439',
 '2440',
 '2478',
 '2513',
 '2546',
 '2548',
 '2549',
 '2552',
 '2569',
 '2583',
 '2584',
 '2587',
 '2590',
 '2598',
 '2603',
 '2607',
 '2612',
 '2614',
 '2618',
 '2621',
 '2625',
 '2647',
 '2657',
 '2662',
 '2668',
 '2689',
 '2712',
 '2715',
 '2716',
 '2722',
 '2724',
 '2727',
 '2737',
 '2745',
 '2754',
 '2762',
 '2796',
 '2800',
 '2805',
 '2827',
 '2846',
 '2950',
 '2967',
 '3021',
 '3037',
 '3038',
 '3039',
 '3040',
 '3041',
 '3046',
 '3051',
 '3052',
 '3058',
 '3067',
 '3084',
 '3085',
 '3087',
 '3095',
 '3149',
 '3180',
 '3187',
 '3346',
 '3374',
 '3479',
 '4897',
 '5164',
 '5566',
 '5912',
 '5920',
 '5943',
 '6349',
 '6353',
 '6359',
 '6360',
 '6373',
 '6403',
 '6453',
 '6560',
 '6562',
 '6573',
 '6580',
 '6621',
 '6627',
 '6628',
 '6632',
 '6640',
 '6669',
 '6767',
 '6773',
 '6987',
 '6993',
 '7015',
 '7028',
 '7271',
 '7414',
 '7766',
 '7774',
 '7776',
 '7784',
 '7969',
 '9033',
 '9090',
 '9367',
 '9469',
 '11493',
 '11753',
 '12363',
 '12368',
 '1238

## Perfil sh

Un perfil es simplemente un conjunto de datos de configuracioń que se carga cuando se inicia IPython. Puede personalizarse un número de perfiles para hacer qye IPython se lleve a cabo de diferentes formas dependiendo de las necesidades de la sesión. Para invocar un perfil específico se utiliza la opción de línea de comando `-p` y se especifica el perfil que se quiere utilizar.

El perfil `sh` es el más utilizado dado que incorpora ciertos elementos de configuración de modo que IPython se convierte en una shell del sistema más amigable.

